Imports

In [ ]:
%pip install -q grad-cam

from utils import imshow, imagenette_outputs, multiple_c_o_m, shift
from ImagenetteDataset import ImagenetteDataset
from load_model import load_model
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import torchvision, torch, torchvision.transforms as T
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from tqdm import tqdm

Configure Size of Imagenette Pictures and PyTorch Device

In [ ]:
#160 uses ~8GB RAM, 320 uses ~24GB RAM, Fullsize not tested
size = 160


if torch.cuda.is_available():
  device = torch.device('cuda:0')
# elif torch.backends.mps.is_available():
#   device = torch.device("mps")
#   import os
#   os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]="1"
else:
  device = "cpu"

print(device)

Download and unpack images

In [ ]:
if not os.path.isfile(f'imagenette2-{size}.tgz'):
    !wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-{size}.tgz
    !tar -xf imagenette2-{size}.tgz



Load Model and target Layers for GradCam

In [ ]:
model, target_layers = load_model('mobilenet', norm_layer=True)


Build our DataLoaders

In [ ]:
bs = 32

trainset = ImagenetteDataset(size, should_normalize=False)
valset = ImagenetteDataset(size, validation=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = bs, shuffle = True)
valloader = torch.utils.data.DataLoader(valset, batch_size = bs, shuffle = False)




Get first Batch for Testing

In [ ]:
data_batch, labels_batch = next(iter(trainloader))
print(data_batch.size())
print(labels_batch.size())
out = torchvision.utils.make_grid(data_batch)

Predict First Batch with our model

In [ ]:
class_names = trainset.classes
print(class_names)
outputs = model(data_batch.to(device))
print(outputs.shape)
preds = imagenette_outputs(outputs)
print(labels_batch)
#print(preds)

In [ ]:
imshow(out, denorm=False)#, title=[class_names[x] for x in preds])

Run Inference on whole trainset

In [ ]:
all_predictions = []
num_correct = 0
with torch.no_grad():
  loop = tqdm(trainloader)
  for idx, (data, labels) in enumerate(loop):
    outputs = model(data.to(device))
    preds = imagenette_outputs(outputs)
    all_predictions.extend(preds)
    corrects = torch.sum(preds == labels.to(device))
    num_correct += corrects
    loop.set_description(f"Processing batch {idx+1}")
    loop.set_postfix(current_accuracy = num_correct.double().item()/(len(labels)*(idx+1)))
    #print(f"Done with batch of size {(len(labels))}")
pred = torch.stack(all_predictions)



In [ ]:
print("Accuracy: {:.4f}".format(num_correct.double()/len(trainset)))

In [ ]:
print(pred.size())
print(pred[0:100])

Run Grad-Cam

In [ ]:

cam = GradCAM(model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available())
#with torch.no_grad():
# 3:23 for 320px and resnet50
# 2:33 for 320px and mobilenetv3
dirname=f'output_{size}'
os.makedirs(dirname, exist_ok=True)
loop = tqdm(trainloader)
examples= []
for batch, (data, labels) in enumerate(loop):
  #make sure we have even number of samples
  if len(labels) % 2 != 0:
    data = data[:-1,:,:,:]
  #Stop after 10 batches to make it quicker
  if batch == 3:
    break
  grayscale_cam = cam(input_tensor=data, targets=None)

  
  threshold = np.quantile(grayscale_cam.flatten(), .85)
  b_mask = np.where(grayscale_cam>threshold, np.ones_like(grayscale_cam), np.zeros_like(grayscale_cam))
  c_o_m = multiple_c_o_m(b_mask)

  imgs_base, imgs_attack = np.array_split(data.cpu().numpy(), 2, axis=0)
  masks_base, masks_attack = np.array_split(b_mask, 2, axis=0)
  c_o_m_base, c_o_m_attack = np.array_split(c_o_m, 2, axis=0)
  offsets = (c_o_m_base - c_o_m_attack).astype(int)
  for base_img, attack_img, base_mask, offset in zip(imgs_base, imgs_attack, masks_base, offsets):
    invariance_adv = np.where(base_mask==True, shift(attack_img, offset), base_img)
    all_examples.append(invariance_adv[:,:,:])
  


Export Pictures to disk

In [ ]:
for idx, img in enumerate(examples):
  img = (img*255).astype(np.uint8)
  img = Image.fromarray(np.uint8(img.transpose(1,2,0)))
  img.save(f"/content/drive/MyDrive/adv_examples_320/{idx}.jpg")

Plot some of the Pictures

In [ ]:
f, xarr = plt.subplots(3,3, figsize=(15,15))
xarr.flatten()
for idx, ax in enumerate(xarr.flatten()):
  ax.imshow(batch_examples[idx].transpose(1,2,0))

In [ ]:
idx = 4
com_b = c_o_m_base[idx]
com_a = c_o_m_attack[idx]
offset = offsets[idx]
base_image = imgs_base[idx]
attack_image = imgs_attack[idx]
print (base_image.shape)
print (attack_image.shape)
fig, ax = plt.subplots(1, 2, figsize=(20, 20))
ax = ax.flatten()
ax[0].imshow(base_image.transpose(1,2,0))
ax[0].scatter(com_b[0], com_b[1], s=size, c='C0', marker='+')
ax[1].imshow(attack_image.transpose(1,2,0))
ax[1].scatter(com_a[0], com_a[1], s=size, c='C1', marker='+')
ax[1].scatter(com_b[0], com_b[1], s=size, c='C0', marker='+')


In [ ]:
print(attack_image.shape)
shifted = shift(attack_image, offset)
#print(offset[::-1])
print (shifted.shape)
plt.imshow(shifted.transpose(1,2,0))

# print(com_b, com_a, offset)
# attack_image_cropped = attack_image[:,39:,11:]
# #plt.imshow(attack_image_cropped.transpose(1,2,0))
# print(attack_image_cropped.shape)
# empty = np.zeros_like(attack_image)
# empty[:,0:121,0:149] = attack_image_cropped
# print(empty.shape)



In [ ]:
invariance_adv = np.where(masks_base[0]==True, shifted, base_image)
plt.imshow(invariance_adv.transpose(1,2,0))

In [ ]:
from scipy.ndimage import shift
print(offsets[11])
print(masked_base[11].shape)
print(offsets[0,0])
print (offsets[:,0])
one_image = imgs_attack[:,:,offsets[:,0]:,offsets[:,1]:]

#shifted = shift(masked_base[11], offsets[11], cval=0)
plt.imshow(attack_patches[4].transpose(1,2,0))
#plt.imshow(masked_base[11].transpose(1,2,0))

In [ ]:
print(normalized.shape)
fig, ax = plt.subplots(2, 4, figsize=(20, 20))
ax = ax.flatten()
for i in range(8):
  idx = random.randint(0, len(masked_images)-1)
  ax[i].imshow(normalized[idx].transpose(0,1))


In [ ]:
c_o_m = multiple_c_o_m(masked_images)

print(c_o_m.shape)
#c_o_m

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(20, 20))
ax = ax.flatten()
for i in range(8):
  idx = random.randint(0, len(masked_images)-1)
  ax[i].imshow(masked_images[idx].transpose(1,2,0))
  ax[i].scatter(c_o_m[idx][0], c_o_m[idx][1], s=size, c='C0', marker='+')
  print(idx, c_o_m[idx])


plt.show()

In [ ]:
#not needed
threshold = np.quantile(gradcam_hm.flatten(), .85)
b_mask = np.where(gradcam_hm>threshold, np.ones_like(gradcam_hm), np.zeros_like(gradcam_hm))
print (b_mask.shape)
img_batch = next(iter(trainloader))[0]
idx = 4
plt.imshow((b_mask[idx]*img_batch[idx].detach().cpu().numpy()).transpose(1,2,0))

Explainability with Pytorch Captum

In [ ]:
%pip install -q git+https://github.com/pytorch/captum.git

from captum.attr import IntegratedGradients, NoiseTunnel
from captum.attr import visualization as viz
from matplotlib.colors import LinearSegmentedColormap

Integrated Gradients

In [ ]:

ig = IntegratedGradients(model)
data, labels = next(iter(trainloader))
idx = 4
input = data[idx].unsqueeze(0).to(device)
label = labels[idx].to(device)
#print (data[0].size())
attributions = ig.attribute(input, target=label, n_steps=100)

default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                 [(0, '#ffffff'),
                                                  (0.25, '#000000'),
                                                  (1, '#000000')], N=256)

_ = viz.visualize_image_attr(np.transpose(attributions.squeeze().cpu().detach().numpy(), (1,2,0)),
                             np.transpose(data[idx].squeeze().cpu().detach().numpy(), (1,2,0)),
                             method='heat_map',
                             cmap=default_cmap,
                             show_colorbar=True,
                             sign='positive',
                             outlier_perc=1)


In [ ]:
imshow(data[idx], denorm=False)

Noise Tunnel for Smooting

In [ ]:
# nt_samples <= 7 for 15GB VRAM 
noise_tunnel = NoiseTunnel(ig)

attributions_ig_nt = noise_tunnel.attribute(input, nt_samples=20, nt_type='smoothgrad_sq', target=label)
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_nt.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      np.transpose(data[idx].squeeze().cpu().detach().numpy(), (1,2,0)),
                                      ["original_image", "heat_map"],
                                      ["all", "positive"],
                                      cmap=default_cmap,
                                      show_colorbar=True)

In [ ]:
#plt.imshow(show_cam_on_image(np.transpose(attributions_ig_nt.squeeze().cpu().detach().numpy(), (1,2,0)),
 #                                     np.transpose(data[idx].squeeze().cpu().detach().numpy(), (1,2,0)), use_rgb=True))

Shift an image

In [ ]:
from PIL import Image
image = Image.open('/Users/dibse/SynologyDrive/Bachelorarbeit/Importance_based_Adversarial_Examples/imagenette2-160/train/n01440764/ILSVRC2012_val_00000293.JPEG')
grayscale_cam = cam(input_tensor=torch.tensor(image).unsqueeze(0), targets=None)